In [47]:
# Import necessary libraries for data retrieval and visualization
import mysql.connector
import pandas as pd
import datetime as dt
import plotly.express as px
from fpdf import FPDF
from pathlib import Path
from constant import connection  # 'connection' holds database connection details

# Set display options for Pandas dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Define the Plotly template for visualization aesthetics
plotly_template = "presentation"


<span style="font-size: 24px; color: black;">**Visualizing Total Sales by Month**</span>

In [48]:
# Establish a buffered cursor for the database connection
mycursor = cnx.cursor(buffered=True)

# SQL query to calculate total sales by date
query = '''
    SELECT o.order_date, ROUND(SUM(oi.list_price * oi.quantity * (1 - oi.discount)), 1) AS total_sales FROM orders o 
    INNER JOIN order_items oi ON o.order_id = oi.order_id 
    GROUP BY order_date 
    ORDER BY order_date
'''

# Reading SQL query results into a Pandas DataFrame
df = pd.read_sql_query(query, cnx)

/var/folders/j3/n_qw7ty16qbc7hzw6mjd45180000gn/T/ipykernel_22681/2917860540.py:15: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [49]:
# Check the data type
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 725 entries, 0 to 724
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   order_date   725 non-null    object 
 1   total_sales  725 non-null    float64
dtypes: float64(1), object(1)
memory usage: 11.5+ KB


In [50]:
# Convert 'order_date' to datatime
df['order_date'] = pd.to_datetime(df['order_date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 725 entries, 0 to 724
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   order_date   725 non-null    datetime64[ns]
 1   total_sales  725 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 11.5 KB


In [51]:
# Set 'order_date' as the DataFrame index
df = df.set_index('order_date')
df.head()

,total_sales
order_date,
2016-01-01,11929.0
2016-01-02,1520.0
2016-01-03,5250.0
2016-01-04,12980.1
2016-01-05,10598.9


In [52]:
# Resample data to calculate total sales monthly
df_monthly = df.resample('M').sum()
df_monthly.head()

,total_sales
order_date,
2016-01-31,215146.3
2016-02-29,156112.4
2016-03-31,180600.4
2016-04-30,167144.1
2016-05-31,205269.6


In [53]:
# Extract month and year information
df_monthly['month'] = df_monthly.index.strftime('%b')
df_monthly['year'] = df_monthly.index.strftime('%Y')
df_monthly.head()

,total_sales,month,year
order_date,,,
2016-01-31,215146.3,Jan,2016
2016-02-29,156112.4,Feb,2016
2016-03-31,180600.4,Mar,2016
2016-04-30,167144.1,Apr,2016
2016-05-31,205269.6,May,2016


In [54]:
# # Create a Plotly bar chart depicting total sales by month
fig = px.bar(df_monthly,
             x='month',
             y='total_sales',
             color='year',
             template=plotly_template,
             barmode='group',
             text='total_sales')

fig.update_layout(
    title='Total Sales by Month',
    xaxis_title='Month',
    yaxis_title='Total Sales ($)',
    yaxis_tickprefix='$',
    font=dict(
        family='Arial',
        size=14,
    ),
    title_font=dict(size=20),             
    xaxis=dict(title_font=dict(size=16)), 
    yaxis=dict(title_font=dict(size=16)), 
    legend=dict(font=dict(size=16)),      
)

fig.show()

<span style="font-size: 24px; color: black;">**Analyzing Most Preferred Product Categories by Customer Purchases**</span>

In [39]:
# SQL query to determine total quantity sold per product category
query = '''
SELECT SUM(quantity) AS total_quantity, category_name
FROM order_items oi
INNER JOIN products p ON oi.product_id = p.product_id
INNER JOIN categories c ON p.category_id = c.category_id
GROUP BY category_name
ORDER BY total_quantity DESC
'''

# Execute the SQL query and fetch the results into a DataFrame
df = pd.read_sql_query(query, cnx)

# Convert 'total_quantity' to integer type for clarity
df["total_quantity"] = df["total_quantity"].astype(int)

df.head()


/var/folders/j3/n_qw7ty16qbc7hzw6mjd45180000gn/T/ipykernel_22681/2668996257.py:12: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,total_quantity,category_name
0,2063,Cruisers Bicycles
1,1755,Mountain Bikes
2,1179,Children Bicycles
3,813,Comfort Bicycles
4,559,Road Bikes


In [40]:
# Create a Plotly bar chart illustrating most preferred categories based on product sales
fig = px.bar(df,
             x='category_name',
             y='total_quantity',
             template=plotly_template,
             text='total_quantity')

fig.update_layout(
    title='Most Preferred Categories by Customers Based on Product Sales',
    xaxis_title='Categories',
    yaxis_title='Total Quantity',
    font=dict(
        family='Arial',
        size=14,
    ),
    title_font=dict(size=20),
    xaxis=dict(title_font=dict(size=16)),
    yaxis=dict(title_font=dict(size=16)),
    legend=dict(font=dict(size=16)),
)

fig.show()


<span style="font-size: 24px; color: black;">**Analyzing Customer Preferences for Product Brands Based on Sales**</span>

In [41]:
# SQL query to determine total quantity sold per brand in each product category
query = '''
SELECT b.brand_name, SUM(quantity) AS total_quantity, c.category_name
FROM order_items oi
INNER JOIN products p ON oi.product_id = p.product_id
INNER JOIN brands b ON b.brand_id = p.brand_id
INNER JOIN categories c ON c.category_id = p.category_id
GROUP BY b.brand_name, c.category_name
ORDER BY total_quantity DESC
'''

# Execute the SQL query and fetch the results into a DataFrame
df = pd.read_sql_query(query, cnx)

# Convert 'total_quantity' to integer type for clarity
df["total_quantity"] = df["total_quantity"].astype(int)

df.head()

/var/folders/j3/n_qw7ty16qbc7hzw6mjd45180000gn/T/ipykernel_22681/3138288196.py:13: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,brand_name,total_quantity,category_name
0,Electra,1329,Cruisers Bicycles
1,Trek,752,Mountain Bikes
2,Electra,747,Children Bicycles
3,Surly,526,Mountain Bikes
4,Electra,524,Comfort Bicycles


In [42]:
# Create a Plotly bar chart illustrating most preferred brands by customers based on product sales
fig = px.bar(df,
             x='brand_name',
             y='total_quantity',
             color='category_name',
             template=plotly_template,
             text='total_quantity')

# Layout
fig.update_layout(
    title='Most Preferred Brands by Customers Based on Product Sales',
    xaxis_title='Brands',
    yaxis_title='Total Quantity',
    font=dict(
        family='Arial',
        size=14,
    ),
    title_font=dict(size=20),             
    xaxis=dict(title_font=dict(size=16)), 
    yaxis=dict(title_font=dict(size=16)), 
    legend=dict(font=dict(size=16)),
    height=1000
)

fig.show()

<span style="font-size: 24px; color: black;">**Comparative Analysis of Store Sales Against Average Sales**</span>

In [29]:
# SQL query to compare total sales of each store against the average sales across all stores
query = '''
    WITH sales AS 
	(SELECT store_name, ROUND(SUM(list_price * quantity * (1 - discount)), 1) AS total_sales
	FROM orders o
	INNER JOIN stores s ON o.store_id = s.store_id
	INNER JOIN order_items oi ON oi.order_id = o.order_id
	GROUP BY store_name)

SELECT *
FROM sales
JOIN (SELECT AVG(total_sales) AS avg_sales FROM sales) avg_sales_table
ON sales.total_sales > avg_sales_table.avg_sales
'''

# Set display format for floats to one decimal place
pd.set_option('display.float_format', lambda x: '%.1f' % x)

# Execute the SQL query and fetch the results into a DataFrame
df = pd.read_sql_query(query, cnx)
df

/var/folders/j3/n_qw7ty16qbc7hzw6mjd45180000gn/T/ipykernel_22681/1663400141.py:16: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,store_name,total_sales,avg_sales
0,Baldwin Bikes,5215751.3,2563038.8


<span style="font-size: 24px; color: black;">**RFM Analysis for Customer Segmentation**</span>

In [55]:
# SQL query to gather necessary data for RFM analysis
query = '''
SELECT o.customer_id, o.order_id, o.order_date, oi.quantity, oi.list_price, oi.discount
FROM orders o
INNER JOIN order_items oi
ON oi.order_id = o.order_id
'''

# Execute the SQL query and fetch the results into a DataFrame
df = pd.read_sql_query(query, cnx)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4722 entries, 0 to 4721
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   customer_id  4722 non-null   int64  
 1   order_id     4722 non-null   int64  
 2   order_date   4722 non-null   object 
 3   quantity     4722 non-null   int64  
 4   list_price   4722 non-null   float64
 5   discount     4722 non-null   float64
dtypes: float64(2), int64(3), object(1)
memory usage: 221.5+ KB


/var/folders/j3/n_qw7ty16qbc7hzw6mjd45180000gn/T/ipykernel_22681/1670707836.py:12: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [56]:
# Data type conversions and data cleaning
date_columns = df.columns[df.columns.str.contains("date")]
df[date_columns] = df[date_columns].apply(pd.to_datetime)

df["order_id"] = df["order_id"].astype(str)
df["customer_id"] = pd.to_numeric(df["customer_id"], errors='coerce')
df["totalPrice"] = df["list_price"] * df["quantity"] * (1 - df["discount"])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4722 entries, 0 to 4721
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   customer_id  4722 non-null   int64         
 1   order_id     4722 non-null   object        
 2   order_date   4722 non-null   datetime64[ns]
 3   quantity     4722 non-null   int64         
 4   list_price   4722 non-null   float64       
 5   discount     4722 non-null   float64       
 6   totalPrice   4722 non-null   float64       
dtypes: datetime64[ns](1), float64(3), int64(2), object(1)
memory usage: 258.4+ KB


In [33]:
df.head(10)

,customer_id,order_id,order_date,quantity,list_price,discount,totalPrice
0,259,1,2016-01-01,1,600.0,0.2,480.0
1,259,1,2016-01-01,2,1800.0,0.1,3348.0
2,259,1,2016-01-01,2,1549.0,0.1,2943.1
3,259,1,2016-01-01,2,600.0,0.1,1140.0
4,259,1,2016-01-01,1,2900.0,0.2,2320.0
5,1212,2,2016-01-01,1,600.0,0.1,558.0
6,1212,2,2016-01-01,2,600.0,0.1,1140.0
7,523,3,2016-01-02,1,1000.0,0.1,950.0
8,523,3,2016-01-02,1,600.0,0.1,570.0
9,175,4,2016-01-03,2,750.0,0.1,1350.0


In [57]:
# Analyzing recency, frequency, and monetary values for RFM segmentation
analysis_date = dt.datetime(2018, 12, 31)
rfm = df.groupby('customer_id').agg({'order_date': lambda order_date: (analysis_date - order_date.max()).days, 
                                     'order_id': lambda order_id: order_id.nunique(), 
                                     'totalPrice': lambda TotalPrice: TotalPrice.sum()}) 
rfm.columns = ['recency', 'frequency', 'monetary']
rfm.head()

,recency,frequency,monetary
customer_id,,,
1,43,3,27888.2
2,266,3,19329.1
3,71,3,24051.5
4,257,3,21150.9
5,258,3,17520.3


In [58]:
# Assigning RFM scores and defining segments based on scores
rfm["recency_score"] = pd.qcut(rfm["recency"], 5, labels=[5, 4, 3, 2, 1])
rfm["frequency_score"] = pd.qcut(rfm["frequency"].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
rfm["monetary_score"] = pd.qcut(rfm["monetary"], 5, labels=[1, 2, 3, 4, 5])
rfm["RF_SCORE"] = rfm["recency_score"].astype(str) + rfm["frequency_score"].astype(str)

df.head()

,customer_id,order_id,order_date,quantity,list_price,discount,totalPrice
0,259,1,2016-01-01,1,600.0,0.2,480.0
1,259,1,2016-01-01,2,1800.0,0.1,3348.0
2,259,1,2016-01-01,2,1549.0,0.1,2943.1
3,259,1,2016-01-01,2,600.0,0.1,1140.0
4,259,1,2016-01-01,1,2900.0,0.2,2320.0


In [59]:
# Mapping RFM scores to customer segments

seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}

rfm['segment'] = rfm['RF_SCORE'].replace(seg_map, regex=True)
rfm.head()
rfm = rfm[["recency", "frequency", "monetary","recency_score" ,"frequency_score","RF_SCORE" , "segment"]]
rfm.head()

,recency,frequency,monetary,recency_score,frequency_score,RF_SCORE,segment
customer_id,,,,,,,
1,43,3,27888.2,5,5,55,champions
2,266,3,19329.1,5,5,55,champions
3,71,3,24051.5,5,5,55,champions
4,257,3,21150.9,5,5,55,champions
5,258,3,17520.3,5,5,55,champions


In [60]:
# Counting customers in each segment and creating a Treemap chart
segment_counts = rfm['segment'].value_counts()
segment_counts_rfm = segment_counts.reset_index()
segment_counts_rfm.columns = ['segment', 'count']
fig = px.treemap(segment_counts_rfm, path=['segment'], values='count', color='segment')

# Update layout and display the Treemap chart
fig.update_layout(margin=dict(l=0, r=0, t=0, b=0))
fig.update_traces(textinfo='label+value', 
                  textfont=dict(size=16),
                  textposition='middle center')
fig.show()

In [38]:
# Saving specific segment data to CSV files
new_df = pd.DataFrame()
new_df["loyal_customer_id"] = rfm[rfm["segment"] == "loyal_customers"].index
new_df["loyal_customer_id"] = new_df["loyal_customer_id"].astype(int)
new_df.to_csv("loyal_customers.csv")
rfm.to_csv("rfm.csv")